In [1]:
#Spectral tools first experiments
#Based on José Luis Notebook on Spectral tools

In [2]:
import numpy as np
from math import pi, sqrt, cos, acos # adding cos and acos
from scipy import linalg as LA
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import ticker, cm
import cmath


In [3]:
##########################################################################################
# Chebyshev Interpolation routines
##########################################################################################

#Identical to José Luis'

def Chebyshev(m,x) :
    
    """
    Returns the value of the T_m(x)
    """
    return cos(m*acos(x))

# Modification:
# Comented out unnecessary variable

def Cheb_spectral_interpol(coeffs,x) :

    """
    Returns the Chebyshev interpolation at x\in[1,1], given the
    Chebishev coefficients coeffs
    """
    n = len(coeffs)
    #EG: this N does nothing in this function.  it is just a bookkeeping device, right?
    # EG:  N labels the x_N point and there are n=N+1 in the set {x_N}.
    #N = n-1
    n_grid = np.arange(0,n)
    
    temp=0
    for k in np.arange(1,n):
        temp += coeffs[k]*Chebyshev(k,x)
    
    interpol = 1./2.*coeffs[0] + temp
    
    return interpol

#########################################################################################
# Chebyshev basic spectral elements
##########################################################################################

##########################################################################################
#Chebyshev-Lobatto
##########################################################################################

#Modification: dictionary description changed

def CL_grid(N) : 
    
    """
    Implement collocation points of the Chebyshev-Lobatto grid
    x_j with j=0,1,...,N
    (following point 4.6.4. in Marcus' notes)
    
    It returns the collocation points
    """
    # EG: This function does not return the Differentiation Matrix, only the collocation points
    n = N+1
    n_grid = np.arange(0, n)
    x_CL = np.cos(pi*n_grid/N)
    
    return x_CL

#Identical to José Luis'

def coeffs_CL(f) :
    
    """ 
    Reads an ndarray with the values of the function at the Chebyshev-Lobatto
    collocation points and return the spectral coefficients
    (following point 4.3.4. in Marcus' notes)
    
    Note: f[N] corresponds to f(-1) and f[0]=1
    """
    
    n = len(f)
    N = n-1
    n_grid = np.arange(0,n)
    
    coeffs = np.zeros(n)
    print(coeffs)
    

    for i in n_grid :
        if i == N :
            temp = 0.
            for k in np.arange(1,N) :
                temp += f[k]*Chebyshev(i,CL_grid(N)[k])
            coeffs[i] = 1./(2.*N)*(f[0] + np.float_power((-1),i)*f[N] + 2.*temp)
           
            
        else :
            temp = 0
            for k in np.arange(1,N) :
                temp +=  f[k]*Chebyshev(i,CL_grid(N)[k])
            coeffs[i] = 1./N*(f[0] + np.float_power((-1),i)*f[N] + 2.*temp)
            
    return coeffs



#########################################################################################
# Spectral-Derivation Matrices
##########################################################################################

# identical to José Luis'

##########################################################################################
#Chebyshev-Lobatto
##########################################################################################


def D1_CL(N) :

    """
    Implement the matrix of the 1st Derivative
    associated with the Chebyshev-Lobatto grid,
    with grid points: x_j with j=0,1,...,N
    (following point 4.6.4. in Marcus' notes)
    
    It returns a tuple with the grid points in the
    first entry and the Differentation Matrix in the second entry
    """
    
    n = N+1
    n_grid = np.arange(0, n)
    x_CL = np.cos(pi*n_grid/N)
    
    kappa = np.ones(n); kappa[0]=2.; kappa[N]=2.
    D1_CL = np.zeros((n,n))

    for i in n_grid :
        #Diagonal values
        D1_CL[0,0] = (2.*N**2+1.)/6.
        D1_CL[N,N] = -(2.*N**2+1.)/6.
        if ((i!=0) & (i!=N)) : D1_CL[i,i] = -x_CL[i]/(2*(1-x_CL[i]**2))
        #Out-of-diagonal values
        for j in n_grid :
            if (j!=i) : D1_CL[i,j] = kappa[i]/kappa[j]* (-1.)**(i-j)/(x_CL[i]-x_CL[j])
            #if (j!=i) : D1_CL[i,j] = (-1)**(i-j)    
    
    return x_CL, D1_CL


    
def D2_CL(N) :

    """
    Implement the matrix of the 2nd Derivative
    associated with the Chebyshev-Lobatto grid,
    with grid points: x_j with j=0,1,...,N
    (following point 4.6.4. in Marcus' notes)
    
    It returns a tuple with the grid points in the
    first entry and the Differentation Matrix in the second entry
    """
    
    n = N+1
    n_grid = np.arange(0, n)
    x_CL = np.cos(pi*n_grid/N)
    
    kappa = np.ones(n); kappa[0]=2.; kappa[N]=2. 
    D2_CL = np.zeros((n,n))

    for i in n_grid :
        #Diagonal values
        D2_CL[0,0] = (N**4-1.)/15.
        D2_CL[N,N] = D2_CL[0,0]
        if ((i!=0) & (i!=N)) : 
            D2_CL[i,i] = -1./(1-x_CL[i]**2)**2 - (N**2-1.)/(3.*(1.-x_CL[i]**2))
        #Out-of-diagonal values
        for j in n_grid :
            if (j!=0) : 
                D2_CL[0,j] = 2./3. * (-1.)**j/kappa[j] * ((2.*N**2+1.)*(1-x_CL[j])-6.)/(1.-x_CL[j])**2
            if (j!=N) : 
                D2_CL[N,j] = 2./3. * (-1.)**(N+j)/kappa[j] * ((2.*N**2+1.)*(1+x_CL[j])-6.)/(1.+x_CL[j])**2
            if ((j!=i) & (i!=0) & (i!=N)) : 
                D2_CL[i,j] = (-1.)**(i-j)/kappa[j] * (x_CL[i]**2+x_CL[i]*x_CL[j]-2.)/((1-x_CL[i]**2)*(x_CL[j]-x_CL[i])**2)  

    return x_CL, D2_CL

##########################################################################################
#Chebyshev-Gauss
##########################################################################################


def D1_CG(N) :

    """
    Implement the matrix of the 1st Derivative
    associated with the Chebyshev-Gauss grid,
    with grid points: x_j with j=0,1,...,N
    (following point 4.6.3. in Marcus' notes)
    
    It returns a tuple with the grid points in the
    first entry and the Differentation Matrix in the second entry
    """
    
    n = N+1
    n_grid = np.arange(0, n)
    x_CG = np.cos((pi*n_grid+0.5*pi)/(N+1))
    D1_CG = np.zeros((n,n))

    for i in n_grid :
        for j in n_grid :
            if (i==j) :   D1_CG[i,j] = (x_CG[i])/(2*(1-(x_CG[i])**2))
            if (i!=j) :   D1_CG[i,j]=(np.float_power((-1),(i-j)))/(x_CG[i]-x_CG[j])*sqrt((1-(x_CG[j])**2)/(1-x_CG[i]**2))
    
    return x_CG, D1_CG 


def D2_CG(N) :
    
    """
    Implement the matrix of the 2nd Derivative
    associated with the Chebyshev-Gauss grid,
    with grid points: x_j with j=0,1,...,N
    (following point 4.6.3. in Marcus' notes)
    
    
    It returns a tuple with the grid points in the
    first entry and the Differentation Matrix in the second entry
    """
    
    n = N+1
    n_grid = np.arange(0, n)
    x_CG = np.cos((pi*n_grid+pi*0.5)/(N+1))
    D2_CG = np.zeros((n,n))

    for i in n_grid :
        for j in n_grid :
            if (i==j) : D2_CG[i,j] = (x_CG[i])**2/(1-(x_CG[i])**2)**2-N*(N+2)/(3*(1-(x_CG[i])**2))
            else : D2_CG[i,j] = (np.float_power((-1),(i-j)))/(x_CG[i]-x_CG[j])*sqrt((1-x_CG[j]**2)/(1-x_CG[i]**2))*(((x_CG[i])/(1-x_CG[i]**2))-(2/(x_CG[i]-x_CG[j])))
    return x_CG, D2_CG   


# we add the corresponding code for left and right radau


##########################################################################################
#Right-Chebyshev-Radau
##########################################################################################

def D1_RCR(N) :

    """
    Implement the matrix of the 1st Derivative
    associated with the Right Chebyshev-Radau grid,
    with grid points: x_j with j=0,1,...,N
    (following points 4.3.1 and 4.6.1 in Marcus' notes)
    
    It returns a tuple with the grid points in the
    first entry and the Differentation Matrix in the second entry
    """
    
    n = N+1
    n_grid = np.arange(0, n)
    x_RCR = np.cos((2*pi*n_grid)/(2*N+1))
    D1_RCR = np.zeros((n,n))

    for i in n_grid :
        for j in n_grid :
            ####### shorthands ########
            A1 = sqrt(2*(1+x_RCR[j]))
            A2 = 1-x_RCR[j]           
            B1 = sqrt(2)*(1-x_RCR[i])*sqrt(1+x_RCR[i])
            C1 = sqrt(1+x_RCR[j])/sqrt(1+x_RCR[i])
            C2 = x_RCR[i]-x_RCR[j]
            ###########################
            if (i==0 and j==0) :   D1_RCR[i,j] = N*(N+1)/3
            if (i==0 and j!=0) :   D1_RCR[i,j] = np.float_power((-1),j)*A1/A2  
            if (i!=0 and j==0) :   D1_RCR[i,j] =  np.float_power((-1),(i+1))/B1  
            if (i!=0 and j==i) :   D1_RCR[i,j] = -1/(2*(1-x_RCR[i]**2))
            if (i!=0 and j!=0 and j!=i) :   D1_RCR[i,j] =  np.float_power((-1),(i-j))*C1/C2                      
    return x_RCR, D1_RCR 


def D2_RCR(N) :

    """
    Implement the matrix of the 2nd Derivative
    associated with the Right Chebyshev-Radau grid,
    with grid points: x_j with j=0,1,...,N
    (following point  4.3.1 and 4.6.1 in Marcus' notes)
    
    It returns a tuple with the grid points in the
    first entry and the Differentation Matrix in the second entry
    """
    
    n = N+1
    n_grid = np.arange(0, n)
    x_RCR = np.cos((2*pi*n_grid)/(2*N+1))
    D2_RCR = np.zeros((n,n))
    
    
    for i in n_grid :
        for j in n_grid :
            ########## shorthands #########
            A1 = 2*sqrt(2)*sqrt(1+x_RCR[j])
            A2 =  3*(1-x_RCR[j])**2
            A3 = N*(N+1)*(1-x_RCR[j])-3
            B1 = 2*x_RCR[i]+1
            B2 = sqrt(2)*((1-x_RCR[i])**2)*(1+x_RCR[i])**(3/2)
            C1 = 2*x_RCR[i]**2-x_RCR[i]+x_RCR[j]-2
            C2 = ((x_RCR[i]-x_RCR[j])**2)*(1-x_RCR[i]**2)
            C3 = sqrt((1+x_RCR[j])/(1+x_RCR[i])) 
            ##############################
            if (i==0 and j==0) :   D2_RCR[i,j] = (N-1)*N*(N+1)*(N+2)/15
            if (i==0 and j!=0) :   D2_RCR[i,j] = (np.float_power((-1),j)*A1*A3)/A2  
            if (i!=0 and j==0) :   D2_RCR[i,j] =  np.float_power((-1),(i+1))*B1/B2 
            if (i!=0 and j==i) :   D2_RCR[i,j] = -N*(N+1)/(3*(1-x_RCR[i]**2)) - x_RCR[i]/(1-x_RCR[i]**2)**2
            if (i!=0 and j!=0 and j!=i) :   D2_RCR[i,j] =  np.float_power((-1),(i-j))*C1*C3/C2  
    return x_RCR, D2_RCR


##########################################################################################
#Left-Chebyshev--Radau
##########################################################################################

def D1_LCR(N) :

    """
    Implement the matrix of the 1st Derivative
    associated with the Left Chebyshev-Radau grid,
    with grid points: x_j with j=0,1,...,N
    (following points 4.3.2 and 4.6.2 in Marcus' notes)
    
    It returns a tuple with the grid points in the
    first entry and the Differentation Matrix in the second entry
    """
    
    n = N+1
    n_grid = np.arange(0, n)
    x_LCR = -np.cos((2*pi*n_grid)/(2*N+1))
    ##########################
    # the right radau grid is needed due to the way the differentiation matrix is expressed in Marcus' notes.
    #  Notice that the right radau grid is not given in the output of this function
    # it is just an internal calculation needed for writing the differentiation matrices
    #########################
    x_RCR =  np.cos((2*pi*n_grid)/(2*N+1))
    D1_LCR = np.zeros((n,n))

    for i in n_grid :
        for j in n_grid :
             ####### shorthands ########
            A1 = sqrt(2*(1+x_RCR[j]))
            A2 = 1-x_RCR[j]           
            B1 = sqrt(2)*(1-x_RCR[i])*sqrt(1+x_RCR[i])
            C1 = sqrt(1+x_RCR[j])/sqrt(1+x_RCR[i])
            C2 = x_RCR[j]-x_RCR[i]
            ###########################
            if (i==0 and j==0) :   D1_LCR[i,j] = -N*(N+1)/3
            if (i==0 and j!=0) :   D1_LCR[i,j] = np.float_power((-1),j+1)*A1/A2  
            if (i!=0 and j==0) :   D1_LCR[i,j] =  np.float_power((-1),i)/B1  
            if (i!=0 and j==i) :   D1_LCR[i,j] = 1/(2*(1-x_RCR[i]**2))
            if (i!=0 and j!=0 and j!=i) :   D1_LCR[i,j] =  np.float_power((-1),(i-j))*C1/C2    
    return x_LCR, D1_LCR 


def D2_LCR(N) :

    """
    Implement the matrix of the 2nd Derivative
    associated with the Left Chebyshev-Radau grid,
    with grid points: x_j with j=0,1,...,N
    (following point  4.3.2 and 4.6.2 in Marcus' notes)
    
    It returns a tuple with the grid points in the
    first entry and the Differentation Matrix in the second entry
    """
    
    n = N+1
    n_grid = np.arange(0, n)
    x_LCR = -np.cos((2*pi*n_grid)/(2*N+1))
    ##########################
    # the right radau grid is needed due to the way the differentiation matrix is expressed in Marcus' notes.
    #  Notice that the right radau grid is not given in the output of this function
    # it is just an internal calculation needed for writing the differentiation matrices
    #########################
    x_RCR =  np.cos((2*pi*n_grid)/(2*N+1))
    D2_LCR = np.zeros((n,n))
    

    for i in n_grid :
        for j in n_grid :
            ########## shorthands #########
            A1 = 2*sqrt(2)*sqrt(1+x_RCR[j])
            A2 =  3*(1-x_RCR[j])**2
            A3 = N*(N+1)*(1-x_RCR[j])-3
            B1 = 2*x_RCR[i]+1
            B2 = sqrt(2)*((1-x_RCR[i])**2)*(1+x_RCR[i])**(3/2)
            C1 = 2*x_RCR[i]**2-x_RCR[i]+x_RCR[j]-2
            C2 = ((x_RCR[i]-x_RCR[j])**2)*(1-x_RCR[i]**2)
            C3 = sqrt((1+x_RCR[j])/(1+x_RCR[i])) 
            ##############################
            if (i==0 and j==0) :   D2_LCR[i,j] = (N-1)*N*(N+1)*(N+2)/15
            if (i==0 and j!=0) :   D2_LCR[i,j] = (np.float_power((-1),j)*A1*A3)/A2  
            if (i!=0 and j==0) :   D2_LCR[i,j] =  np.float_power((-1),(i+1))*B1/B2 
            if (i!=0 and j==i) :   D2_LCR[i,j] = -N*(N+1)/(3*(1-x_RCR[i]**2)) - x_RCR[i]/(1-x_RCR[i]**2)**2
            if (i!=0 and j!=0 and j!=i) :   D2_LCR[i,j] =  np.float_power((-1),(i-j))*C1*C3/C2      
    return x_LCR, D2_LCR

In [4]:
#########################################################
# Spectrum calculator
#########################################################

# Modifications:  B is not used and N=n-1 instead of N= n/2 - 1
def Spectrum(M) :
    """
    Implement Spectrum of a matrix M.
    Returns a tuple with the real and imaginary part of the eigenvalues respectively.
    """
    from scipy import linalg as LA
    
    ### 1. Preparation of tools
    sizeM = np.shape(M)
    if  np.shape(M)[0]!= np.shape(M)[1]:
        print("Non-square Matrix!!! ")
    else : 
        n = np.shape(M)[0]
                       
        eigenvalues_M = LA.eigvals(M)
        eigenvalues_M_Re = eigenvalues_M.real
        eigenvalues_M_Im = eigenvalues_M.imag
        return eigenvalues_M
    
    # Modifications:  following José Luis'
def SpectrumAlt(L,B) :
    """
    Implement Spectrum of a matrix M.
    Returns a tuple with the real and imaginary part of the eigenvalues respectively.
    """
    from scipy import linalg as LA
    
    ### 1. Preparation of tools
    sizeL = np.shape(L)
    if  np.shape(L)[0]!= np.shape(L)[1]:
        print("Non-square Matrix!!! ")
    else : 
        n = np.shape(L)[0]
    N = n/2 - 1
    # EG: should't this be  n//2 -1 so that it gives an integer?
    # EG: this N is different from Rodrigo's notation/convention N=n-1
   
    ### 2. Calculation of the Spectrum (eigenvalues)
    #eigenvalues_L, eigenvectors_L = LA.eig(L,B)
    eigenvalues_L = LA.eigvals(L,B)

    eigenvalues_L_Re = eigenvalues_L.real
    eigenvalues_L_Im = eigenvalues_L.imag
    
    return eigenvalues_L
        

In [5]:
#################################################
########### pseudospectrum calculator ###########
#################################################

In [6]:
def Pseudospectrum_standard(L,B,xmin,xmax,ymin,ymax,Nxgrid,Nygrid) :
    """
    Implement Pseudo-Spectrum of a matrix L
    """
    from scipy import linalg as LA
    
    ### 1. Preparation of tools 
    sizeL = np.shape(L)
    if  np.shape(L)[0]!= np.shape(L)[1]:
        print("Non-square Matrix!!! ")
    else : 
        n = np.shape(L)[0]
    N = n/2 - 1
   
    ### 2. Calculation of the Spectrum (eigenvalues)
    #eigenvalues_L, eigenvectors_L = LA.eig(L,B)
    eigenvalues_L = LA.eigvals(L,B)


    eigenvalues_L_Re = eigenvalues_L.real
    eigenvalues_L_Im = eigenvalues_L.imag

    
    ### 3. Evaluation of the Pseudospectrum
    
    ### 3.1 Grid for Pseudospectrum calculation
    [X,Y] = np.mgrid[xmin:xmax:Nxgrid*1j,ymin:ymax:Nygrid*1j]

    Z = X + 1j*Y

    ### 3.2 Construction of the "height function" given by the min of the SVP
    Id =  np.eye(n)
    Sigma_min = np.zeros((Nxgrid,Nygrid))
    

    for i in np.arange(0, Nxgrid):
        for j in np.arange(0, Nygrid):
            L_shift = L - Z[i,j]*B*Id            
            Sigma_min[i,j] = min(np.linalg.svd(L_shift, full_matrices=True)[1]) 

    return eigenvalues_L, Sigma_min
#    
#    ### 3.3 Graphical output
#    fig = plt.figure()
#    ax = fig.add_subplot(111)
#    ax.plot(eigenvalues_L_Re, eigenvalues_L_Im, '+', markersize=1)
#     
#    if fl == "f" :
#        CS = ax.contourf(X,Y,Sigma_min,heights,locator=ticker.LogLocator(),linewidth=0.5)
#    elif fl == "c" : 
#        CS = ax.contour(X,Y,Sigma_min,heights,locator=ticker.LogLocator(),linewidth=0.5)
#    else :
#        print("\n Pseudospectrum output: \n No 'contour/filled' version could be identified.\n Filled version is assumed.\n")
#        CS = ax.contourf(X,Y,Sigma_min,heights,locator=ticker.LogLocator(),linewidth=0.5)
#
#    CB = fig.colorbar(CS)
#    ax.set_xlabel(r'$\mathrm{Re}(\omega_n)$')
#    ax.set_ylabel(r'$\mathrm{Im}(\omega_n)$')    
#    f = mticker.ScalarFormatter(useOffset=False, useMathText=True)
#    g = lambda x,pos : "${}$".format(f._formatSciNotation('%10e' % x))
#    ax.xaxis.set_major_formatter(mticker.FuncFormatter(g))
#    ax.yaxis.set_major_formatter(mticker.FuncFormatter(g))
#    
#    fig.suptitle(r'Pseudospectrum using the $L^2$-inner-product' )
#    ax.axis('scaled')
#    #ax.axis('equal','datalim')
#    ax.axis([xmin,xmax,ymin,ymax])
#    ax.grid()   
#    #ax.set_xlim(xmin,xmax)
#    #ax.set_ylim(ymin,ymax)
#    fig.savefig("Figures_abc/Pseudospectrum-Standard-Inner-Product"+"_N"+str(N)+"_Nx"+str(Nxgrid)+"_Ny"+str(Nygrid)+".pdf")
#
#
#    fig.show()
#    print("\n N =\n",N)
#    

In [7]:
def GraphicalOutput(eigenvalues_L,Sigma_min, N, label_gram, xmin,xmax,ymin,ymax,Nxgrid,Nygrid,heights, fl)  :

        
    ### 3.1 Grid for Pseudospectrum calculation
    [X,Y] = np.mgrid[xmin:xmax:Nxgrid*1j,ymin:ymax:Nygrid*1j]

    Z = X + 1j*Y
    
    eigenvalues_L_Re = eigenvalues_L.real
    eigenvalues_L_Im = eigenvalues_L.imag

    ### 3.3 Graphical output
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(eigenvalues_L_Re, eigenvalues_L_Im, '+', markersize=1)
     
    if fl == "f" :
        CS = ax.contourf(X,Y,Sigma_min,heights,locator=ticker.LogLocator(),linewidth=0.5)
    elif fl == "c" : 
        CS = ax.contour(X,Y,Sigma_min,heights,locator=ticker.LogLocator(),linewidth=0.5)
    else :
        print("\n Pseudospectrum output: \n No 'contour/filled' version could be identified.\n Filled version is assumed.\n")
        CS = ax.contourf(X,Y,Sigma_min,heights,locator=ticker.LogLocator(),linewidth=0.5)
    
    CB = fig.colorbar(CS)
    ax.set_xlabel(r'$\mathrm{Re}(\omega_n)$')
    ax.set_ylabel(r'$\mathrm{Im}(\omega_n)$')    
    f = mticker.ScalarFormatter(useOffset=False, useMathText=True)
    g=f
    # formatSciNotation does not work (in new versions of one of the pacakges, matplotlib? python3.9?)
    #g = lambda x,pos : "${}$".format(f._formatSciNotation('%10e' % x))
    ax.xaxis.set_major_formatter(mticker.FuncFormatter(g))
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(g))
     
    if label_gram =="st" :
        fig.suptitle(r'Pseudospectrum using the $L^2$-inner-product' )
        ax.axis('scaled')
        #ax.axis('equal','datalim')
        ax.axis([xmin,xmax,ymin,ymax])
        ax.grid()   
        #ax.set_xlim(xmin,xmax)
        #ax.set_ylim(ymin,ymax)
        fig.savefig("Figures_abc/Pseudospectrum-Standard-Inner-Product"+"_N"+str(N)+"_Nx"+str(Nxgrid)+"_Ny"+str(Nygrid)+".pdf")
    
        fig.show()
        print("\n N =\n",N)
    
    else :    
        
        fig.suptitle(r'Pseudospectrum using Gram Matrix = ' + str(label_gram) )
        ax.axis('scaled')
        #ax.axis('equal','datalim')
        ax.axis([xmin,xmax,ymin,ymax])
        ax.grid()   
        #ax.set_xlim(xmin,xmax)
        #ax.set_ylim(ymin,ymax)
        fig.savefig("Figures_abc/Pseudospectrum"+"GramMatrix" +str(label_gram) +"_N"+str(N)+"_Nx"+str(Nxgrid)+"_Ny"+str(Nygrid)+".pdf")


        fig.show()
        print("\n N =\n",N)


In [8]:
def Kdelta(i,j):
    if (i==j):
        KroneckerDeltaReturn=1
    else:
        KroneckerDeltaReturn=0
    
    return KroneckerDeltaReturn

In [9]:
def CNmuMatrix(mu) :
    """ 
    Reads an ndarray with the values of the weight function mu at the Chebyshev-Lobatto
    collocation points and returns the matrix C^N_mu
    (following equation C14 in Jaramillo-PanossoMacedo-Al-Sheikh paper)
    """
    n = len(mu)
    N = n-1
    n_grid = np.arange(0, n)
    x_grid = CL_grid(N)
    alpha = np.ones(n)
    alpha[0]=2
    alpha[N]=2
    #list of indices in the sum
    # we add one because we want to include the point floor(N/2) in the arrange.
    Nhalffloor_grid = np.arange(1, int(np.floor(N/2)+1))
    #print("n =" ,  n)
    #print("N =" ,  N)
    #print("Nhalffloor_grid =" ,  Nhalffloor_grid)
    CNmuMatrix=np.zeros((n,n))
    for i in n_grid :
        for j in n_grid :
            if (i==j) :  
                #we construct a list of the elements to be summed up using List comprehension
                ExpressionInSummedUpInListForm = [(Chebyshev(2*k,x_grid[i]))*((2-Kdelta(2*k,N))/(4*(k**2)-1)) for k  in Nhalffloor_grid]
                CNmuMatrix[i,j] = ((2*mu[i])/(alpha[i]*N))*(1-sum(ExpressionInSummedUpInListForm))
            else :               
                CNmuMatrix[i,j] = 0
              
    return CNmuMatrix

In [10]:
def FunctionAtCollocationPoints(f,n):
    """ 
    Gives the function f evaluated at the x[i] collocation point of a Lobatto grid of size n
    """
    N = n-1
    n_grid = np.arange(0, n)
    x_grid = CL_grid(N)
    fFunctionAtx= f(x_grid) 
    return fFunctionAtx 

In [11]:
def InterpolationMatrixI(nbar,n):
    """ 
    Gives the interpolation matrix for the transition between 
    the interpolant vectors f_Nbar and the f_N with resolutions Nbar and N respectively
    (following equation C18 in Jaramillo-PanossoMacedo-Al-Sheikh paper)
    """
    ####
    Nbar = nbar -1
    #nbar = Nbar + 1
    nbar_grid = np.arange(0, nbar)
    xbar_grid = CL_grid(Nbar)
    #####
    N = n-1
    #n = N + 1
    n_grid = np.arange(0, n)
    x_grid = CL_grid(N)
    #####
    alpha = np.ones(n)
    alpha[0]=2
    alpha[N]=2
    IndexSummation_grid = np.arange(1, n)
    #####
    Interpolant_Matrix=np.zeros((nbar,n))
    for ibar in nbar_grid :
        for i in n_grid :
            ExpressionListToBeSummed = [(2-Kdelta(j,N))*Chebyshev(j,xbar_grid[ibar])*Chebyshev(j,x_grid[i]) for j in IndexSummation_grid ]
            Interpolant_Matrix[ibar,i] = (1/(N*alpha[i]))*(1+sum(ExpressionListToBeSummed))
    return Interpolant_Matrix 

In [12]:
def Matrix_Interpolated(M) :
    """ 
    Reads a matrix M with associated resolution n_high and returns
    the interpolated matrix   I^t M I with resolution n_low = n_high/2
    (following equation C21 in Jaramillo-PanossoMacedo-Al-Sheikh paper)
    """
    n_high=len(M)
    n_low = int(n_high/2)
    I = InterpolationMatrixI(n_high,n_low)
    It = InterpolationMatrixI(n_high,n_low).transpose()
    temp=np.dot(M,I)
    M_interpolated = np.dot(It,temp)
              
    return M_interpolated

In [13]:
def ConjugateTranspose(A):
    return A.conj().T

def Dagger(Gram_Matrix,M):
    temp=np.dot(ConjugateTranspose(M),Gram_Matrix)
    return np.dot(np.linalg.inv(Gram_Matrix),temp)

def EigenValues_gram(M,Gram_Matrix):
    M_dagger = Dagger(Gram_Matrix,M)
    MdaggerM = np.dot(M_dagger,M)
    eigenvalues_MdaggerM = LA.eigvals(MdaggerM)
    return eigenvalues_MdaggerM 

def GramMatrix_abc_not_interpolated(w):
    # For the abc operator the Gram Matrix is simple
    # This needs to be changed for a more complicated case such as poschl-teller
    GramMatrix=CNmuMatrix(w)
    return GramMatrix

def GramMatrix_abc(w):
    # For the abc operator the Gram Matrix is simple
    # This needs to be changed for a more complicated case such as poschl-teller
    GramMatrix=Matrix_Interpolated(CNmuMatrix(w))
    return GramMatrix


In [14]:
def Pseudospectrum_gram_norm(L,B,xmin,xmax,ymin,ymax,Nxgrid,Nygrid,Gram_Matrix) :
    """
    Implement Pseudo-Spectrum of a matrix L
    """
    from scipy import linalg as LA
    
    ### 1. Preparation of tools 
    sizeL = np.shape(L)
    if  np.shape(L)[0]!= np.shape(L)[1]:
        print("Non-square Matrix!!! ")
    else : 
        n = np.shape(L)[0]
    N = n/2 - 1
    
    #
    #SizeGram =np.shape(Gram_Matrix)
    #if SizeGram != sizeL:
        #print ("Size of Gram Matrix should coincide with Size of L")
    #else:
   
    ### 2. Calculation of the Spectrum (eigenvalues)
    #eigenvalues_L, eigenvectors_L = LA.eig(L,B)
    eigenvalues_L = LA.eigvals(L,B)


    eigenvalues_L_Re = eigenvalues_L.real
    eigenvalues_L_Im = eigenvalues_L.imag

    
    ### 3. Evaluation of the Pseudospectrum
    
    ### 3.1 Grid for Pseudospectrum calculation
    [X,Y] = np.mgrid[xmin:xmax:Nxgrid*1j,ymin:ymax:Nygrid*1j]

    Z = X + 1j*Y

    ### 3.2 Construction of the "height function" given by the min of the SVP
    Id =  np.eye(n)
    Sigma_min = np.zeros((Nxgrid,Nygrid))
    

    for i in np.arange(0, Nxgrid):
        for j in np.arange(0, Nygrid):
            L_shift = L - Z[i,j]*B*Id            
            Sigma_min[i,j] = np.min(np.sqrt(np.abs(EigenValues_gram(L_shift,Gram_Matrix)))) 
            #min(np.linalg.svd(L_shift, full_matrices=True)[1]) 
    
    return eigenvalues_L, Sigma_min

#####################    
#    ### 3.3 Graphical output
#    fig = plt.figure()
#    ax = fig.add_subplot(111)
#    ax.plot(eigenvalues_L_Re, eigenvalues_L_Im, '+', markersize=1)
#     
#    if fl == "f" :
#        CS = ax.contourf(X,Y,Sigma_min,heights,locator=ticker.LogLocator(),linewidth=0.5)
#    elif fl == "c" : 
#        CS = ax.contour(X,Y,Sigma_min,heights,locator=ticker.LogLocator(),linewidth=0.5)
#    else :
#        print("\n Pseudospectrum output: \n No 'contour/filled' version could be identified.\n Filled version is assumed.\n")
#        CS = ax.contourf(X,Y,Sigma_min,heights,locator=ticker.LogLocator(),linewidth=0.5)
#
#    CB = fig.colorbar(CS)
#    ax.set_xlabel(r'$\mathrm{Re}(\omega_n)$')
#    ax.set_ylabel(r'$\mathrm{Im}(\omega_n)$')    
#    f = mticker.ScalarFormatter(useOffset=False, useMathText=True)
#    g = lambda x,pos : "${}$".format(f._formatSciNotation('%10e' % x))
#    ax.xaxis.set_major_formatter(mticker.FuncFormatter(g))
#    ax.yaxis.set_major_formatter(mticker.FuncFormatter(g))
#    
#    fig.suptitle(r'Pseudospectrum using Gram Matrix = ' + str(label_gram) )
#    ax.axis('scaled')
#    #ax.axis('equal','datalim')
#    ax.axis([xmin,xmax,ymin,ymax])
#    ax.grid()   
#    #ax.set_xlim(xmin,xmax)
#    #ax.set_ylim(ymin,ymax)
#    fig.savefig("Figures_abc/Pseudospectrum"+"GramMatrix" +str(label_gram) +"_N"+str(N)+"_Nx"+str(Nxgrid)+"_Ny"+str(Nygrid)+".pdf")
#
#    fig.show()
#    print("\n N =\n",N)
#############################

In [15]:
def Diagonal_Random_Matrix(n) :
    D = np.random.rand(n,1)
    Id = np.eye(n)
    DR = Id * D 
    #visualisation_matrix(DR)
    #DR = np.zeros(n,n)    
    #for i in range(n):
    #    DR[i,i] = D[i,i] 
    return DR

In [16]:
def SavingDataFunction(eigenvalues_L,Sigma_min, FileName) :
    
    """
    Saves data to two files. The first one with the eigenvalues and the other one with the pseudospectrum data.
    """
    np.savetxt('Data_abc/Spectrum_'+str(FileName)+'.txt', eigenvalues_L.view(float))
    np.savetxt('Data_abc/Pseudospectrum_'+str(FileName)+'.txt', Sigma_min)                                                                                                        
    
def ReadingDataFunction(FileName) :
    
    """
    Reads data from a canonically named pair of files.
    """
    SpectrumData = np.loadtxt('Data_abc/Spectrum_'+str(FileName)+'.txt').view(complex)
    PseudospectrumData = np.loadtxt('Data_abc/Pseudospectrum_'+str(FileName)+'.txt')
    return SpectrumData, PseudospectrumData